<a href="https://colab.research.google.com/github/cglima/ftc_dados_python/blob/main/ciclo_05_Transforma%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ciclo 05 : Transformação de Dados

## Gráficos em Python

In [1]:
# from google import colab as cl
import pandas as pd
import re
from haversine import haversine

In [2]:
# file_upload = cl.files.upload()
# url = 'https://raw.githubusercontent.com/cglima/ftc_dados_python/main/dataset/train.csv'
df = pd.read_csv('dataset/train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/train.csv'

In [ ]:
df1 = df.copy()
# excluir as linhas com os registros vazios (NaN)
# idade dos entregadores vazia
linhas_selecionadas = df1['Delivery_person_Age'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

# ID vazio
linhas_selecionadas = df1['ID'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

# City vazia
linhas_selecionadas = df1['City'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

# Festival
linhas_selecionadas = df1['Festival'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

# Road_traffic_density vazia
linhas_selecionadas = df1['Road_traffic_density'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()

# 1. convertendo a coluna Age de texto para numero
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype(int)

# 2. convertendo a coluna Rating de texto para numero decimal (float)
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype(float)

# 3. convertendo a coluna order_date de texto para data
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], format='%d-%m-%Y')

# 4. convertendo multiple_deliveries de texto para numero inteiro (int)
linhas_selecionadas = df1['multiple_deliveries'] != 'NaN '
df1 = df1.loc[linhas_selecionadas, :].copy()
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype(int)

# 5. removendo os espaços dentro de strings/texto/object
# df1.iloc[0, 1]
df1.loc[0, 'City']

df1.loc[:, 'ID'] = df1.loc[:, 'ID'].str.strip()
df1.loc[:, 'Delivery_person_ID'] = df1.loc[:, 'Delivery_person_ID'].str.strip()
df1.loc[:, 'Road_traffic_density'] = df1.loc[:, 'Road_traffic_density'].str.strip()
df1.loc[:, 'Type_of_order'] = df1.loc[:, 'Type_of_order'].str.strip()
df1.loc[:, 'Type_of_vehicle'] = df1.loc[:, 'Type_of_vehicle'].str.strip()
df1.loc[:, 'City'] = df1.loc[:, 'City'].str.strip()
df1.loc[:, 'Festival'] = df1.loc[:, 'Festival'].str.strip()

# 6. removendo o texto de números
df1['Time_taken(min)'] = df1['Time_taken(min)'].apply( lambda x: x.split('(min)')[1])
# df1['Time_taken(min)'] = df1['Time_taken(min)'].apply(lambda x: re.findall( r'\d+', x))
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype(int)



# 7. resetando o index do dataset
df1 = df1.reset_index(drop=True)

In [ ]:
df1.head()

# Visão: Empresa

## 1. Quantidade de pedidos por dia.

In [ ]:
#colunas
cols = ['ID', 'Order_Date']

#selecao de linhas
df_aux = df1.loc[:, cols].groupby('Order_Date').count().reset_index()
df_aux.head()

# desenhar o gráfico de linhas
#Plotly
import plotly.express as px

px.bar(df_aux, x= 'Order_Date', y='ID')

## 2. Quantidade de pedidos por semana.

In [ ]:
# criar a coluna de semana
df1['week_of_year'] = df1['Order_Date'].dt.strftime('%U')

df_aux = df1.loc[:, ['ID', 'week_of_year']].groupby( 'week_of_year').count().reset_index()

px.line(df_aux, x='week_of_year', y='ID')

## 3. Distribuição dos pedidos por tipo de tráfego.

In [ ]:
df_aux = df1.loc[:, ['ID', 'Road_traffic_density']].groupby('Road_traffic_density').count().reset_index()
df_aux['entregas_perc'] = df_aux['ID'] / df_aux['ID'].sum()
df_aux.head()

px.pie(df_aux, values='entregas_perc', names='Road_traffic_density')

## 4. Comparação do volume de pedidos por cidade e tipo de tráfego.


In [ ]:
df_aux = df1.loc[:, ['ID', 'City', 'Road_traffic_density']].groupby(['City', 'Road_traffic_density']).count().reset_index()

px.scatter(df_aux, x='City', y='Road_traffic_density', size='ID', color='City' )

## 5. A quantidade de pedidos por entregador por semana.

In [ ]:
# quantidade de pedidos por semana  / numero unico de entregadores por semana
df_aux01 = df1.loc[:, ['ID', 'week_of_year']].groupby('week_of_year').count().reset_index()
df_aux02 = df1.loc[:, ['Delivery_person_ID', 'week_of_year']].groupby('week_of_year').nunique().reset_index()

df_aux = pd.merge(df_aux01, df_aux02, how='inner')
df_aux['order_by_deliver'] = df_aux['ID'] / df_aux['Delivery_person_ID']

px.line(df_aux, x='week_of_year', y='order_by_deliver')

## 6. A localização central de cada cidade por tipo de tráfego.


In [ ]:
import folium
df_aux = df1.loc[:, ['City', 'Road_traffic_density', 'Delivery_location_latitude', 'Delivery_location_longitude']].groupby(['City', 'Road_traffic_density' ]).median().reset_index()

map = folium.Map(zoom_start=11)

#folium faz a marcação no mapa, tem que passar a latitude e a longitude
#folium.Marker([latitude, longitude])
for index, location_info in df_aux.iterrows():
  folium.Marker([location_info['Delivery_location_latitude'],
                 location_info['Delivery_location_longitude']],
                 popup=location_info[['City', 'Road_traffic_density']]).add_to(map)

map

# Visão: Entregadores

## 1. A menor e maior idade dos entregadores.

In [ ]:
print(f" A menor idade é: {df1.loc[:, 'Delivery_person_Age'].min()}")
print(f" A maior idade é: {df1.loc[:, 'Delivery_person_Age'].max()}")

## 2. A pior e a melhor condição de veículos.

In [ ]:
print(f" A pior condição de veículo é: {df1.loc[:, 'Vehicle_condition'].min()}")
print(f" A melhor condição de veículo é: {df1.loc[:, 'Vehicle_condition'].max()}")

## 3. A avaliação média por entregador.

In [ ]:
df_avg_ratings_per_deliver = df1.loc[:, ['Delivery_person_ID','Delivery_person_Ratings']].groupby('Delivery_person_ID').mean().reset_index()
df_avg_ratings_per_deliver

## 4. A avaliação média e o desvio padrão por tipo de tráfego.

In [ ]:
# df1.loc[linhas, clounas]
# funçao de agregação : agg({'coluna_que_vai_sofrer_operacao' : ['tipo_de operação']})
df_avg_std_rating_by_traffic = ( df1.loc[:, ['Delivery_person_Ratings','Road_traffic_density']]
                            .groupby('Road_traffic_density')
                            .agg({'Delivery_person_Ratings': ['mean', 'std']}) )

# mudança de nome das colunas
df_avg_std_rating_by_traffic.columns = ['delivery_mean', 'delivery_std']

# reset index
df_avg_std_rating_by_traffic = df_avg_std_rating_by_traffic.reset_index()
df_avg_std_rating_by_traffic

## 5. A avaliação média e o desvio padrão por condições climáticas.

In [ ]:
df_avg_std_rating_by_weather = ( df1.loc[:, ['Delivery_person_Ratings','Weatherconditions']]
                            .groupby('Weatherconditions')
                            .agg({'Delivery_person_Ratings': ['mean', 'std']}) )

# mudança de nome das colunas
df_avg_std_rating_by_weather.columns = ['delivery_mean', 'delivery_std']

# reset index
df_avg_std_rating_by_weather = df_avg_std_rating_by_weather.reset_index()
df_avg_std_rating_by_weather

## 6. Os 10 entregadores mais rápidos por cidade.

In [ ]:
df2 = (df1.loc[:, ['Delivery_person_ID', 'City', 'Time_taken(min)']]
                            .groupby(['City','Delivery_person_ID']).min()
                            .sort_values(['City', 'Time_taken(min)'], ascending=True)
                            .reset_index())

df_aux01 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_aux02 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_aux03 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df_faster_deliveries = pd.concat( [df_aux01, df_aux02, df_aux03]).reset_index(drop=True)
df_faster_deliveries

## 7. Os 10 entregadores mais lentos por cidade.

In [ ]:
df2 = (df1.loc[:, ['Delivery_person_ID', 'City', 'Time_taken(min)']]
                            .groupby(['City','Delivery_person_ID']).max()
                            .sort_values(['City', 'Time_taken(min)'], ascending=False)
                            .reset_index())

df_aux01 = df2.loc[df2['City'] == 'Metropolitian', :].head(10)
df_aux02 = df2.loc[df2['City'] == 'Urban', :].head(10)
df_aux03 = df2.loc[df2['City'] == 'Semi-Urban', :].head(10)

df_lower_deliveries = pd.concat( [df_aux01, df_aux02, df_aux03]).reset_index(drop=True)
df_lower_deliveries

# Visão: Restaurantes

## 1. A quantidade de entregadores únicos.

In [ ]:
delivery_unique = len(df1.loc[:, 'Delivery_person_ID'].unique())

print(f'O numero de entregadores únicos é: {delivery_unique}')

## 2. A distância média dos resturantes e dos locais de entrega.

In [ ]:
cols = ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']

df1['distance'] = df1.loc[:, cols].apply(lambda x:
                          haversine(
                              (x['Restaurant_latitude'], x['Restaurant_longitude']),
                              (x['Delivery_location_latitude'], x['Delivery_location_longitude'])), axis=1)

avg_distance = df1['distance'].mean()
print(f'A distancia media das entregas é: {avg_distance:.2f} KM')

## 3.O tempo médio e o desvio padrão de entrega por cidade.

In [ ]:
cols = ['City', 'Time_taken(min)']
df_aux = df1.loc[:, cols].groupby( 'City' ).agg( {'Time_taken(min)': ['mean', 'std']} )

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()

df_aux

## 4. O tempo médio e o desvio padrão de entrega por cidade e tipo de pedido.

In [ ]:
cols = ['City', 'Time_taken(min)', 'Type_of_order']
df_aux = df1.loc[:, cols].groupby( ['City', 'Type_of_order'] ).agg( {'Time_taken(min)': ['mean', 'std']} )

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()

df_aux

## 5. O tempo médio e o desvio padrão de entrega por cidade e tipo de tráfego.

In [ ]:
cols = ['City', 'Time_taken(min)', 'Road_traffic_density']
df_aux = df1.loc[:, cols].groupby( ['City', 'Road_traffic_density'] ).agg( {'Time_taken(min)': ['mean', 'std']} )

df_aux.columns = ['avg_time', 'std_time']

df_aux = df_aux.reset_index()

df_aux

## 6. O tempo médio de entrega durantes os Festivais.

In [ ]:
cols = ['Time_taken(min)', 'Festival']
df_aux = df1.loc[:, cols].groupby( 'Festival' ).agg( {'Time_taken(min)': ['mean', 'std']} )

df_aux.columns = ['avg_time', 'std_time']
df_aux = df_aux.reset_index()

linhas_selecionadas = df_aux['Festival'] == 'Yes'
df_aux = df_aux.loc[linhas_selecionadas, :]

df_aux